## Imports

In [1]:
from keras.models import Sequential, Model
from keras.layers import ZeroPadding2D, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization, Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import Adam
import numpy as np
import pandas as pd

/home/callum/anaconda2/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set the path of the data
PATH = "~/.kaggle/competitions/digit-recognizer"

In [3]:
# Import dataset
train = pd.read_csv(PATH + "/train.csv")
test = pd.read_csv(PATH + "/test.csv")

## Preprocessing

In [4]:
# We will resize the vectors into 28x28x1 arrays
# example = X_train.values[1]
# np.resize(example, (28,28, 1)).shape

In [5]:
X_train = []
y_train = []

for index, row in train.iterrows():
    label = row["label"]
    row = row.drop(["label"], axis=0)
    image = np.resize(row.values, (28, 28, 1))
    entry = { 'label': label, 'image': image}
#     data.append(entry)
    X_train.append(image)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
# train_processed = pd.DataFrame(data=data); train_processed.head()

In [6]:
# data = []
X_test = []

for index, row in test.iterrows():
    image = np.resize(row.values, (28, 28, 1))
    entry = {'image': image}
    X_test.append(image)
#     data.append(entry)
    
X_test = np.array(X_test)
# test_processed = pd.DataFrame(data=data); test_processed.head()

In [7]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(n_values=10)
y_train_encoded = enc.fit_transform(y_train.reshape(-1, 1))

## On to the Model

In [8]:
inputs = Input(shape=(28,28,1,))

x = ZeroPadding2D((1,1))(inputs)
x = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
x = ZeroPadding2D((1,1))(x)
x = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
x = ZeroPadding2D((1,1))(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = ZeroPadding2D((1,1))(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
# Flatten and dense layers
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(.5)(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [9]:
train_pct = .75
split_pnt = int(train_pct * len(X_train))
X_val, y_val = X_train[split_pnt:], y_train_encoded[split_pnt:]
X_train, y_train = X_train[:split_pnt], y_train_encoded[:split_pnt]
print(len(X_train), len(X_val))

31500 10500


In [10]:
model.fit(X_train, y_train, batch_size=64, validation_data=(X_val, y_val), epochs=3)


Train on 31500 samples, validate on 10500 samples
Epoch 1/3
31500/31500 [==============================] - 19s 606us/step - loss: 0.3441 - acc: 0.8964 - val_loss: 0.1481 - val_acc: 0.9570
Epoch 2/3
31500/31500 [==============================] - 17s 541us/step - loss: 0.0958 - acc: 0.9722 - val_loss: 0.0486 - val_acc: 0.9838
Epoch 3/3
31500/31500 [==============================] - 17s 536us/step - loss: 0.0643 - acc: 0.9821 - val_loss: 0.0433 - val_acc: 0.9877


In [11]:
model.fit(X_train, y_train, batch_size=64, validation_data=(X_val, y_val), epochs=8)

Train on 31500 samples, validate on 10500 samples
Epoch 1/8
31500/31500 [==============================] - 17s 531us/step - loss: 0.0533 - acc: 0.9847 - val_loss: 0.0720 - val_acc: 0.9771
Epoch 2/8
31500/31500 [==============================] - 17s 535us/step - loss: 0.0485 - acc: 0.9858 - val_loss: 0.0474 - val_acc: 0.9874
Epoch 3/8
31500/31500 [==============================] - 17s 535us/step - loss: 0.0403 - acc: 0.9881 - val_loss: 0.0377 - val_acc: 0.9901
Epoch 4/8
31500/31500 [==============================] - 17s 535us/step - loss: 0.0349 - acc: 0.9893 - val_loss: 0.0285 - val_acc: 0.9930
Epoch 5/8
31500/31500 [==============================] - 17s 534us/step - loss: 0.0270 - acc: 0.9916 - val_loss: 0.0369 - val_acc: 0.9895
Epoch 6/8
31500/31500 [==============================] - 17s 533us/step - loss: 0.0227 - acc: 0.9930 - val_loss: 0.0291 - val_acc: 0.9916
Epoch 7/8
31500/31500 [==============================] - 17s 533us/step - loss: 0.0220 - acc: 0.9929 - val_loss: 0.0311 - 

In [14]:
model.save("models/mnist.h5")

In [15]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
preds = model.predict(X_test)

In [24]:
data = []
for i in range(0, len(preds)):
    entry = { 'ImageId': i+1, 'Label': np.argmax(preds[i]) }
    data.append(entry)
submit = pd.DataFrame(data=data)

In [25]:
submit.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [26]:
submit.to_csv('mnist_submit.csv', index=False)